#  Сверточные нейронные сети
Реализация обучения модели на датасете из игральных карт

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pandas as pd
from PIL import Image

In [3]:
torch.__version__
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti with Max-Q Design'

In [4]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False


img_transforms = transforms.Compose([
    transforms.Resize((64,64)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])


train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 

batch_size=64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

for batch_ndx, sample in enumerate(train_data_loader):
    print(sample[0].shape)
    print(batch_ndx)

torch.Size([64, 3, 64, 64])
0
torch.Size([64, 3, 64, 64])
1
torch.Size([64, 3, 64, 64])
2
torch.Size([64, 3, 64, 64])
3
torch.Size([64, 3, 64, 64])
4
torch.Size([64, 3, 64, 64])
5
torch.Size([64, 3, 64, 64])
6
torch.Size([64, 3, 64, 64])
7
torch.Size([64, 3, 64, 64])
8
torch.Size([64, 3, 64, 64])
9
torch.Size([64, 3, 64, 64])
10
torch.Size([64, 3, 64, 64])
11
torch.Size([64, 3, 64, 64])
12
torch.Size([64, 3, 64, 64])
13
torch.Size([64, 3, 64, 64])
14
torch.Size([64, 3, 64, 64])
15
torch.Size([64, 3, 64, 64])
16
torch.Size([64, 3, 64, 64])
17
torch.Size([64, 3, 64, 64])
18
torch.Size([64, 3, 64, 64])
19
torch.Size([64, 3, 64, 64])
20
torch.Size([64, 3, 64, 64])
21
torch.Size([64, 3, 64, 64])
22
torch.Size([64, 3, 64, 64])
23
torch.Size([64, 3, 64, 64])
24
torch.Size([64, 3, 64, 64])
25
torch.Size([64, 3, 64, 64])
26
torch.Size([64, 3, 64, 64])
27
torch.Size([64, 3, 64, 64])
28
torch.Size([64, 3, 64, 64])
29
torch.Size([64, 3, 64, 64])
30
torch.Size([64, 3, 64, 64])
31
torch.Size([64, 3,

In [5]:
file_path_test ="./cards.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

In [6]:
len(data["labels"].unique())


53

In [7]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=53):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [8]:
cnnnet = CNNNet()

----

In [9]:
cnnnet(torch.rand(1, 3,224, 224, requires_grad=True))[0].shape

torch.Size([53])

In [10]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        if (acc< num_correct / num_examples):
                acc = num_correct / num_examples
                
        if (19< epoch):
                acc = num_correct / num_examples
                torch.save(model,f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [11]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=32
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
device = torch.device("cuda") 


In [14]:
torch.cuda.is_available()

True

In [12]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [13]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=21, device=device)

Epoch: 1, Training Loss: 3.57, Validation Loss: 2.45, accuracy = 0.22
Epoch: 2, Training Loss: 2.48, Validation Loss: 1.82, accuracy = 0.34
Epoch: 3, Training Loss: 2.16, Validation Loss: 1.73, accuracy = 0.42
Epoch: 4, Training Loss: 1.89, Validation Loss: 1.26, accuracy = 0.61
Epoch: 5, Training Loss: 1.63, Validation Loss: 1.09, accuracy = 0.66
Epoch: 6, Training Loss: 1.45, Validation Loss: 0.95, accuracy = 0.74
Epoch: 7, Training Loss: 1.38, Validation Loss: 0.94, accuracy = 0.74
Epoch: 8, Training Loss: 1.24, Validation Loss: 0.91, accuracy = 0.76
Epoch: 9, Training Loss: 1.20, Validation Loss: 0.76, accuracy = 0.80
Epoch: 10, Training Loss: 1.14, Validation Loss: 0.77, accuracy = 0.80
Epoch: 11, Training Loss: 1.07, Validation Loss: 0.67, accuracy = 0.80
Epoch: 12, Training Loss: 1.01, Validation Loss: 0.62, accuracy = 0.83
Epoch: 13, Training Loss: 0.97, Validation Loss: 0.64, accuracy = 0.81
Epoch: 14, Training Loss: 0.90, Validation Loss: 0.67, accuracy = 0.82
Epoch: 15, Trai

In [31]:
cnnnet.eval()
#model.half()
x = torch.rand(1, 3,224, 224, requires_grad=True).to(device)
torch_out = cnnnet(x)

In [17]:
# Сохранение модели в формате ONNX
torch.onnx.export(cnnnet,                   # Модель
                  torch.randn(1, 3, 224, 224).to(device), # Пример входных данных
                  "модель.onnx",            # Путь для сохранения
                  export_params=True,       # Сохранять ли параметры модели
                  opset_version=11,         # Версия ONNX
                  do_constant_folding=True, # Оптимизировать ли константные узлы
                  input_names=['input'],    # Имена входных узлов
                  output_names=['output'],  # Имена выходных узлов
                  dynamic_axes={'input': {0: 'batch_size'}, # Динамические оси
                                'output': {0: 'batch_size'}})

In [32]:
import onnx
import onnxruntime
import numpy as np

In [22]:
onnx_model = onnx.load("./model.onnx")

In [41]:


def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()

    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()





из перебранных мной ace of clubs он угадал 4 из 5,интересно что он везде угдал масть но в перой картинке предположи что это король вместо туза

In [59]:
predicted_class = predict_image("model.onnx", "./test/ace of clubs/5.jpg", img_transforms, device)
print("Предсказанный класс:", predicted_class)

class_names = train_data.classes
print("Имя класса :",class_names[predicted_class])

Предсказанный класс: 0
Имя класса : ace of clubs


In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import onnxruntime

# Функция для предсказания класса изображения
def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()
    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()

# Путь к вашей модели ONNX
model_path = "model.onnx"

# Путь к вашему тестовому набору данных
test_data_path = "./test/"


# Преобразования изображений
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [78]:


# Инициализация списка для хранения предсказанных классов и истинных меток
predicted_classes = []
true_labels = []

# Пройти по всем изображениям в тестовом наборе данных
for root, dirs, files in os.walk(test_data_path):
    for file in files:
        if file.endswith(".jpg"):  # Проверка, что файл - изображение
            image_path = os.path.join(root, file)
            true_label = os.path.basename(root)  # Извлечение истинного класса из пути
            predicted_class = predict_image(model_path, image_path, img_transforms, device)
            predicted_classes.append(predicted_class)
            true_labels.append(true_label)


In [80]:
true_labels

['ace of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of diamonds',
 'ace of diamonds',
 'ace of diamonds',
 'ace of diamonds',
 'ace of diamonds',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'eight of clubs',
 'eight of clubs',
 'eight of clubs',
 'eight of clubs',
 'eight of clubs',
 'eight of diamonds',
 'eight of diamonds',
 'eight of diamonds',
 'eight of diamonds',
 'eight of diamonds',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of spades',
 'eight of spades',
 'eight of spades',
 'eight of spades',
 'eight of spades',
 'five of clubs',
 'five of clubs',
 'five of clubs',
 'five of clubs',
 'five of clubs',
 'five of diamonds',
 'five of diamonds',
 'five of diamonds',
 'five of diamonds',
 'five of diamonds',
 'five of hearts',
 'five of hearts

In [81]:
predicted_classes

[21,
 0,
 0,
 0,
 0,
 1,
 3,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 7,
 4,
 4,
 5,
 6,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 32,
 7,
 8,
 33,
 16,
 8,
 8,
 9,
 9,
 9,
 9,
 34,
 11,
 10,
 10,
 9,
 10,
 11,
 11,
 11,
 11,
 11,
 41,
 12,
 12,
 37,
 12,
 13,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 15,
 0,
 15,
 15,
 15,
 29,
 16,
 16,
 16,
 16,
 17,
 30,
 22,
 17,
 17,
 17,
 2,
 18,
 18,
 18,
 30,
 19,
 32,
 19,
 32,
 37,
 20,
 4,
 37,
 20,
 21,
 21,
 21,
 21,
 21,
 22,
 17,
 17,
 22,
 22,
 23,
 23,
 23,
 17,
 23,
 24,
 11,
 24,
 24,
 24,
 28,
 37,
 25,
 25,
 29,
 26,
 26,
 26,
 26,
 35,
 27,
 26,
 27,
 18,
 27,
 28,
 25,
 28,
 28,
 28,
 29,
 29,
 29,
 16,
 30,
 30,
 30,
 30,
 18,
 30,
 31,
 31,
 31,
 9,
 31,
 32,
 32,
 32,
 32,
 32,
 33,
 33,
 33,
 33,
 33,
 34,
 34,
 34,
 34,
 34,
 35,
 35,
 35,
 33,
 35,
 36,
 36,
 36,
 36,
 36,
 26,
 21,
 37,
 37,
 0,
 38,
 38,
 38,
 38,
 38,
 42,
 39,
 39,
 39,
 39,
 40,
 37,
 40,
 40,
 40,
 41,
 4,
 41,
 44,
 41,
 4

In [82]:
untocken_classes = []
for i in predicted_classes:
    untocken_classes.append(class_names[i])


In [83]:
untocken_classes

['king of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of clubs',
 'ace of diamonds',
 'ace of spades',
 'ace of diamonds',
 'ace of diamonds',
 'ace of diamonds',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of hearts',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'ace of spades',
 'eight of clubs',
 'eight of clubs',
 'eight of spades',
 'eight of clubs',
 'eight of clubs',
 'eight of diamonds',
 'eight of hearts',
 'eight of diamonds',
 'eight of diamonds',
 'eight of diamonds',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of hearts',
 'eight of spades',
 'eight of spades',
 'eight of spades',
 'queen of spades',
 'eight of spades',
 'five of clubs',
 'seven of clubs',
 'jack of clubs',
 'five of clubs',
 'five of clubs',
 'five of diamonds',
 'five of diamonds',
 'five of diamonds',
 'five of diamonds',
 'seven of diamonds',
 'five of spades',
 'five of hearts

In [84]:

# Вычисление метрик
accuracy = accuracy_score(true_labels, untocken_classes)
precision = precision_score(true_labels, untocken_classes, average='weighted')
recall = recall_score(true_labels, untocken_classes, average='weighted')
f1 = f1_score(true_labels, untocken_classes, average='weighted')

# Вывод метрик
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7622641509433963
Precision: 0.8069856244384546
Recall: 0.7622641509433963
F1 Score: 0.7588128852279795


---------------------------------------


In [48]:
model=CNNNet()   
x=torch.rand(1,3,224,224)
traced_cpu = torch.jit.trace(model,x)
torch.jit.save(traced_cpu, "cpu.pt")

c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\jit\_trace.py:1102: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 53 / 53 (100.0%)
Greatest absolute difference: 0.008638938656076789 at index (0, 44) (up to 1e-05 allowed)
Greatest relative difference: 12.27697950560337 at index (0, 1) (up to 1e-05 allowed)
  _check_trace(


In [ ]:
model = torch.jit.load("cpu.pt")

In [ ]:
torch.load("./model11_0.7433962264150943.pt")

реализация сохранения модели селанная на занятии

----------